<a href="https://colab.research.google.com/github/martinmedice/Learning_Data_Science/blob/main/Regress%C3%A3o_com_teste_de_diferentes_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas
# ---- Basics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# ---- Cronometrar o tempo do colab
import time

# ---- Pré-processamento
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

# ---- Possíveis Modelos utilizados

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor, LassoLars, BayesianRidge
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor

# ---- Cálculo das Métricas
from sklearn import metrics


# Tirar o chato do Warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

% matplotlib inline

start = time.time()


In [ ]:
car = pd.read_csv('Car.csv', sep=';')

In [ ]:
car.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,wheelbase,carlength,carwidth,...,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,89,168,64,...,2548,130,3,3,9,111,5000,21,27,13495
1,2,3,alfa-romero stelvio,gas,std,two,convertible,89,168,64,...,2548,130,3,3,9,111,5000,21,27,16500
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,95,171,66,...,2823,152,3,3,9,154,5000,19,26,16500
3,4,2,audi 100 ls,gas,std,four,sedan,100,177,66,...,2337,109,3,3,10,102,5500,24,30,13950
4,5,2,audi 100ls,gas,std,four,sedan,99,177,66,...,2824,136,3,3,8,115,5500,18,22,17450


In [ ]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   car_ID            205 non-null    int64 
 1   symboling         205 non-null    int64 
 2   CarName           205 non-null    object
 3   fueltype          205 non-null    object
 4   aspiration        205 non-null    object
 5   doornumber        205 non-null    object
 6   carbody           205 non-null    object
 7   wheelbase         205 non-null    int64 
 8   carlength         205 non-null    int64 
 9   carwidth          205 non-null    int64 
 10  carheight         205 non-null    int64 
 11  curbweight        205 non-null    int64 
 12  enginesize        205 non-null    int64 
 13  boreratio         205 non-null    int64 
 14  stroke            205 non-null    int64 
 15  compressionratio  205 non-null    int64 
 16  horsepower        205 non-null    int64 
 17  peakrpm         

In [ ]:

car = pd.get_dummies(data=car, columns=['CarName','fueltype','aspiration','doornumber','carbody'])

In [ ]:
car.head()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,...,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon
0,1,3,89,168,64,49,2548,130,3,3,...,1,1,0,0,1,1,0,0,0,0
1,2,3,89,168,64,49,2548,130,3,3,...,1,1,0,0,1,1,0,0,0,0
2,3,1,95,171,66,52,2823,152,3,3,...,1,1,0,0,1,0,0,1,0,0
3,4,2,100,177,66,54,2337,109,3,3,...,1,1,0,1,0,0,0,0,1,0
4,5,2,99,177,66,54,2824,136,3,3,...,1,1,0,1,0,0,0,0,1,0


In [ ]:
X = car[car.columns[(car.columns != 'price') & (car.columns != 'car_ID')]]
y =car['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

print(len(X_train))

164


In [ ]:
car.columns

Index(['car_ID', 'symboling', 'wheelbase', 'carlength', 'carwidth',
       'carheight', 'curbweight', 'enginesize', 'boreratio', 'stroke',
       ...
       'fueltype_gas', 'aspiration_std', 'aspiration_turbo', 'doornumber_four',
       'doornumber_two', 'carbody_convertible', 'carbody_hardtop',
       'carbody_hatchback', 'carbody_sedan', 'carbody_wagon'],
      dtype='object', length=174)

In [ ]:
scaler = StandardScaler()
colunas_scaler =['symboling','wheelbase','carlength','carwidth','carheight',
                 'curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg']
                 
scaler.fit(X_train[colunas_scaler])
X_train = X_train.reset_index().drop('index',axis=1)
X_test = X_test.reset_index().drop('index',axis=1)
y_train = y_train.reset_index().drop('index',axis=1)
y_test = y_test.reset_index().drop('index',axis=1)


X_train[colunas_scaler] = pd.DataFrame(scaler.transform(X_train[colunas_scaler]),columns=colunas_scaler)
X_train.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon
0,-1.474241,0.796025,1.114101,0.414951,1.641789,0.852706,0.266108,1.370107,-0.158425,-0.051546,...,1,1,0,1,0,0,0,0,0,1
1,-0.673660,1.460391,0.954805,0.877890,1.237505,1.257531,0.522823,1.370107,-0.156432,2.683408,...,0,0,1,1,0,0,0,0,1,0
2,0.126921,-0.698800,-0.717804,-0.973865,-0.379633,-1.102019,-0.760755,-0.729870,-0.158425,-0.300178,...,1,1,0,0,1,0,0,1,0,0
3,0.927503,-0.200525,0.078677,-0.047988,-0.783917,-0.093813,0.382797,1.370107,-0.156432,-0.300178,...,1,1,0,0,1,0,0,1,0,0
4,0.927503,-0.366617,-0.239915,-0.047988,0.833220,-0.647073,-0.760755,-0.729870,-0.158425,3.180672,...,0,1,0,1,0,0,0,0,1,0


In [ ]:
X_test[colunas_scaler] = pd.DataFrame(scaler.transform(X_train[colunas_scaler]),columns=colunas_scaler)
X_test.head()

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,...,fueltype_gas,aspiration_std,aspiration_turbo,doornumber_four,doornumber_two,carbody_convertible,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon
0,-1.853910,-16.345293,-13.850641,-30.409861,-21.142957,-5.009824,-3.018310,-4.152608,-0.164722,-2.550683,...,1,1,0,1,0,0,0,0,1,0
1,-1.212979,-16.234947,-13.863329,-30.195548,-21.306402,-5.009044,-3.012319,-4.152608,-0.164718,-1.870686,...,1,1,0,0,1,1,0,0,0,0
2,-0.572049,-16.593571,-13.996549,-31.052798,-21.960186,-5.013593,-3.042275,-8.562511,-0.164722,-2.612501,...,0,0,1,1,0,0,0,0,0,1
3,0.068881,-16.510812,-13.933111,-30.624173,-22.123632,-5.011649,-3.015587,-4.152608,-0.164718,-2.612501,...,0,1,0,1,0,0,0,0,1,0
4,0.068881,-16.538398,-13.958486,-30.624173,-21.469848,-5.012716,-3.042275,-8.562511,-0.164722,-1.747050,...,1,1,0,0,1,0,0,0,1,0


In [ ]:
# Treinamento e validação dos Modelos

models = {
    'LinearReg': LinearRegression(),
    'RidgeReg': Ridge(),
    'LassoReg': Lasso(),
    'ElasticNetReg': ElasticNet(),
    'HuberReg': HuberRegressor(),
    'LassoCV': LassoCV(),
    'ElasticNetCV': ElasticNetCV(),
    'LassoLars': LassoLars(),
    'BayesianRidge': BayesianRidge(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'SVM_RBF':SVR(kernel='rbf'),
    'SVM_POLY':SVR(kernel='poly'),
    'SVM_LINEAR':SVR(kernel='linear',max_iter=10**4),
    'SVM_SIG':SVR(kernel='sigmoid'),
    'SGDRegressor':SGDRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    'AdaBoostRegressor':AdaBoostRegressor(),
    'GradientBoostingRegressor':GradientBoostingRegressor(),
    'ExtraTreesRegressor':ExtraTreesRegressor()
}

for model in models.values():
    model.fit(X_train, y_train);

eval = []
nome = []
r2 = []
MAE = []
for name, model in models.items():
    y_pred = model.predict(X_test)
    nome.append(name)
    eval.append(metrics.mean_squared_error(y_test,y_pred))
    r2.append(metrics.r2_score(y_test,y_pred))
    MAE.append(metrics.mean_absolute_error(y_test,y_pred))
    print('-------------------------')
    print(name + ":\n R² : {:.4f}\n MSE: {:.4F}\n MAE: {:.4F}".format(metrics.r2_score(y_test,y_pred),metrics.mean_squared_error(y_test,y_pred),metrics.mean_absolute_error(y_test,y_pred)))
    print('-------------------------')

-------------------------
LinearReg:
 R² : -183.2225
 MSE: 7336161368.5258
 MAE: 81529.3683
-------------------------
-------------------------
RidgeReg:
 R² : -68591.9778
 MSE: 2731529807112.8276
 MAE: 1524256.8202
-------------------------
-------------------------
LassoReg:
 R² : -13811.7497
 MSE: 550055394100.0876
 MAE: 736147.1900
-------------------------
-------------------------
ElasticNetReg:
 R² : -21168.5083
 MSE: 843018409258.7961
 MAE: 874545.9569
-------------------------
-------------------------
HuberReg:
 R² : -152.3914
 MSE: 6108398787.6063
 MAE: 77939.6575
-------------------------
-------------------------
LassoCV:
 R² : -78.2062
 MSE: 3154171822.9656
 MAE: 55796.9835
-------------------------
-------------------------
ElasticNetCV:
 R² : -257.9734
 MSE: 10312916103.9195
 MAE: 101333.9520
-------------------------
-------------------------
LassoLars:
 R² : -54.2951
 MSE: 2201977656.7794
 MAE: 46546.6205
-------------------------
-------------------------
BayesianRid

In [ ]:
# Tabela de Métricas por Modelo

teste = pd.DataFrame({'Modelo': nome, 'MSE': eval, 'R2': r2, 'MAE':MAE})
teste.nsmallest(18,['MAE'])

,Modelo,MSE,R2,MAE
17,GradientBoostingRegressor,3.755467e+07,0.056944,3.614997e+03
16,AdaBoostRegressor,4.013527e+07,-0.007859,3.851224e+03
13,SVM_SIG,4.001465e+07,-0.004830,4.211645e+03
10,SVM_RBF,4.010826e+07,-0.007181,4.252129e+03
18,ExtraTreesRegressor,3.893015e+07,0.022403,4.272305e+03
9,DecisionTreeRegressor,7.092223e+07,-0.780968,5.576732e+03
12,SVM_LINEAR,8.920408e+07,-1.240054,7.017184e+03
11,SVM_POLY,1.953308e+08,-3.905061,1.243658e+04
7,LassoLars,2.201978e+09,-54.295097,4.654662e+04
5,LassoCV,3.154172e+09,-78.206179,5.579698e+04


In [ ]:
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

reg = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1)
reg.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [ ]:
# Fine Tuning 

parameters = {'loss':['squared_loss','huber'],
               'penalty':['l2', 'l1', 'elasticnet'],
               'alpha':[10**-7,10**-6,10**-4,1, 10, 0.002],
               'l1_ratio':[0.15, 0.3, 0.5],
               'max_iter':[10**3]
        }
reg = GridSearchCV(SGDRegressor(), parameters,n_jobs=-1)
reg.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
Reg_Best = reg.best_estimator_
Reg_Best

GradientBoostingRegressor(learning_rate=0.04, max_depth=6, n_estimators=1500,
                          subsample=0.9)

In [ ]:
# Cálculo das Melhores métricas atingidas no Mini-Case

Reg_Best.fit(X_train,y_train)
y_pred = Reg_Best.predict(X_test)
print('-------------------------')
print(name + ":\n R² : {:.4f}\n MSE: {:.4F}\n MAE: {:.4F}".format(metrics.r2_score(y_test,y_pred),metrics.mean_squared_error(y_test,y_pred),metrics.mean_absolute_error(y_test,y_pred)))
print('-------------------------')

-------------------------
ExtraTreesRegressor:
 R² : -0.0704
 MSE: 42625891.1374
 MAE: 3660.0366
-------------------------


In [ ]:
# Criando uma tabela com todos os valores e seus preditores

df_total_final = car.copy()
car[colunas_scaler] = pd.DataFrame(scaler.transform(car[colunas_scaler]),columns=colunas_scaler)
df_total_final['previsto'] = Reg_Best.predict(car[car.columns[(car.columns != 'price') & (car.columns != 'car_ID')]])
df_total_final[['price','previsto']]


,price,previsto
0,13495,8832.047063
1,16500,10402.504304
2,16500,8347.753871
3,13950,8603.347944
4,17450,8494.399457
...,...,...
200,16845,8467.195906
201,19045,8501.729540
202,21485,8674.583054
203,22470,8598.355709
